In [1]:
import logging
logging.basicConfig()
from kazoo.client import KazooClient

In [2]:
class Animal: 
    def __init__(self, name: str, hostPort: int, root: str, partySize: int):
        self.name = name
        self.root = root
        self.animalPath = f'{root}/{name}'
        self.partySize = partySize
        self.zk = KazooClient(hosts=f'127.0.0.1:{hostPort}')
        self.zk.start()

        @self.zk.ChildrenWatch('/zoo')
        def watch_node(children):
            print(f'Children are now: {children}')
            if len(children) < self.partySize:
                print('Waiting for the others.')
            elif len(children) == self.partySize:
                print('Zoo is full.')

    def enter(self):
        self.zk.create(self.animalPath, ephemeral=True)
        print(f'{self.name} entered.')

    def leave(self):
        self.zk.delete(self.animalPath)
        self.zk.stop()

In [3]:
zk = KazooClient()
zk.start()

In [18]:
zk.create(path="/zoo", value=b"first_version")

'/zoo'

In [4]:
animal = Animal('human1', 2181, '/zoo', 2)

Children are now: []
Waiting for the others.


In [5]:
animal.enter()

human1 entered.
Children are now: ['human1']
Waiting for the others.


In [6]:
animal2 = Animal('human2', 2181, '/zoo', 2)

Children are now: ['human1']
Waiting for the others.


In [7]:
animal2.enter()

human2 entered.
Children are now: ['human2', 'human1']
Zoo is full.
Children are now: ['human2', 'human1']
Zoo is full.


In [8]:
animal2.leave()

Children are now: ['human1']Children are now: ['human1']
Waiting for the others.

Waiting for the others.


In [9]:
animal.leave()

Children are now: []
Waiting for the others.


In [10]:
zk.stop()
zk.close()